<a href="https://colab.research.google.com/github/vektor8891/llm/blob/main/projects/28_langchain_rag/28_langchain_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install -qq wget
# ! pip install -qq langchain
# ! pip install -qq langchain-ibm
# ! pip install -qq langchain-community
# ! pip install -qq langchain-huggingface
# ! pip install -qq chromadb
# ! pip install -qq ibm-watsonx-ai

# Summarize Private Documents Using RAG, LangChain, and LLMs

## Preprocessing

In [2]:
import wget

filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

file downloaded


In [3]:
with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability.
Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest.
Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy.
Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our practices. We report any potential violations of 

## Splitting the document into chunks

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

16


## Embedding and storing

In [5]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

document ingested


## LLM model construction

In [6]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from google.colab import userdata
from ibm_watsonx_ai.credentials import Credentials
from ibm_watsonx_ai.foundation_models import Model
from langchain_ibm.llms import WatsonxLLM

model_id = 'google/flan-ul2'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

flan_ul2_llm = WatsonxLLM(
    model_id=model_id,
    url=userdata.get("WATSONX_URL"),
    apikey=userdata.get('IBM_CLOUD_API_KEY'),
    project_id=userdata.get("WATSONX_PROJECT_ID"),
    params=parameters
)

/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'google/flan-ul2' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


## Integrating LangChain

In [7]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

{'query': 'what is mobile policy?',
 'result': 'The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance. Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations. Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device. Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces. Cost Management: Keep personal phone usage separate from company accounts and reimburse the co

In [8]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

{'query': 'Can you summarize the document for me?',
 'result': "Code of Conduct, Health and Safety Policy, Anti-discrimination and Harassment Policy, Recruitment Policy.. I think that's it.. I'm not sure.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.."}

In [9]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="refine",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "What should I do if I find a lost mobile phone?"
qa.invoke(query)

{'query': 'What should I do if I find a lost mobile phone?',
 'result': 'Immediately report any lost or stolen mobile devices to the IT department or your supervisor. Consequences: Non-compliance with this policy may lead to disciplinary actions, including the potential loss of mobile phone privileges. The Mobile Phone Policy is aimed at promoting the responsible and secure use of mobile devices in line with legal and ethical standards. Every employee is expected to comprehend and abide by these guidelines. Regular reviews of the policy ensure its ongoing alignment with evolving technology and security best practices. --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# model_id = 'meta-llama/llama-3-3-70b-instruct'
model_id = 'google/flan-t5-xxl'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

llama_3_llm = WatsonxLLM(
    model_id=model_id,
    url=userdata.get("WATSONX_URL"),
    apikey=userdata.get('IBM_CLOUD_API_KEY'),
    project_id=userdata.get("WATSONX_PROJECT_ID"),
    params=parameters
)

qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

/usr/local/lib/python3.11/dist-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


{'query': 'Can you summarize the document for me?',
 'result': "Our Code of Conduct outlines the fundamental principles and ethical standards that guide every member of our organization. We are committed to maintaining a workplace that is built on integrity, respect, and accountability. Integrity: We hold ourselves to the highest ethical standards. This means acting honestly and transparently in all our interactions, whether with colleagues, clients, or the broader community. We respect and protect sensitive information, and we avoid conflicts of interest. Respect: We embrace diversity and value each individual's contributions. Discrimination, harassment, or any form of disrespectful behavior is unacceptable. We create an inclusive environment where differences are celebrated and everyone is treated with dignity and courtesy. Accountability: We take responsibility for our actions and decisions. We follow all relevant laws and regulations, and we strive to continuously improve our pract

## Dive deeper

In [12]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 return_source_documents=False)
query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': 'No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are owned or leased, to maintain the condition and cleanliness of these vehicles. So no eating..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..'}

### Using prompt template

In [13]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [14]:
qa = RetrievalQA.from_chain_type(llm=llama_3_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(),
                                 chain_type_kwargs=chain_type_kwargs,
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

{'query': 'Can I eat in company vehicles?',
 'result': 'No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are owned or leased, to maintain the condition and cleanliness of these vehicles. Enforcement and Consequences: All employees and visitors are expected to adhere to this policy. Non-compliance may lead to appropriate disciplinary action, which could include fines, or, in the case of employees, possible termination of employment. Review of Policy: This policy will be reviewed periodically to ensure its alignment with evolving legal requirements and best practices for maintaining a healthy and safe workplace. We appreciate your cooperation in maintaining a smoke-free and safe environment for all.'}

### Make the conversation have memory

In [ ]:
query = "What I cannot do in it?"
qa.invoke(query)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm,
                                           chain_type="stuff",
                                           retriever=docsearch.as_retriever(),
                                           memory = memory,
                                           get_chat_history=lambda h : h,
                                           return_source_documents=False)

In [ ]:
history = []

query = "What is mobile policy?"
result = qa.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

history.append((query, result["answer"]))

In [ ]:
query = "List points in it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

history.append((query, result["answer"]))

In [ ]:
query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])